# Advent of Code 2025

> Computer science is no more about computers than astronomy is about telescopes.
>
-- Edsger W. Dijkstra

This year there will only be [12 problems](https://adventofcode.com/2025/about#faq_num_days) rather than 24. It won't be any less enjoyable!

## Imports and definitions

In [1]:
from urllib import request
from itertools import accumulate, combinations
from functools import cache
from math import lcm


def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2025/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

In [2]:
def hits(move, start):
    revolutions = abs(move) // 100
    signed_mod = move % 100 if move >= 0 else -(-move % 100)
    return revolutions + (start != 0 and not 0 < start + signed_mod < 100)


rotations = [int(x.strip('R').replace('L', '-')) for x in aocin(1).split()]

A = sum(x % 100 == 0 for x in accumulate(rotations, initial=50))
assert A == 992

A = sum(hits(r, s % 100) for r, s in zip(rotations, accumulate(rotations, initial=50)))
assert A == 6133

## [Day 2: Gift Shop](https://adventofcode.com/2025/day/2)

An extremely interesting problem for day 2!

Essentially, the problem gives us a list of pairs of numbers $L = ((a_0, b_0), (a_1, b_1), \dots)$ and asks us to find the sum of all numbers in each range that are the result of the concatenation of certain number of "blocks" of (at least two) repeating digits. Those are _invalid ids_.

So, for example, $121212$ would be invalid, because it contains the block $12$ three times. In the first part of the problem, only numbers made of exactly two of those blocks are taken into account, and in the second part the restriction is lifted. We'll analyze the second part as it's a straightforward extension of the first one.

### Solution

Let $N$ be a number of $\delta_N$ digits. $N$ is invalid as per the definition above if and only if for some $k$, with $k | \delta_N$, it can be written as the concatenation of $\delta_N / k$ blocks of $k$ digits each.
Let this block be composed by digits $c_1 c_2 \dots c_k$, where juxtaposition denotes concatenation in base 10 representation.
This would mean that $N$ could be written as:

$$
N = c_1 c_2 \dots c_k \dots c_1 c_2 \dots c_k \dots \dots c_1 c_2 \dots c_k
$$

where the block $c_1 c_2 \dots c_k$ is repeated $\delta_N / k$ times. But this means that:

$$
N = c_1 c_2 \dots c_k \frac{10^{\delta_N} - 1}{10^k - 1}
$$

That is, $N$ is a multiple of a "magic number"

$$
\mu(\delta_N, k) = \frac{10^{\delta_N} - 1}{10^k - 1}
$$

Our second observation is that it's easy to sum the multiples of $n$ from $a$ to $b$, with $a \le b$.
The least of such multiples is $\left \lceil a / n \right \rceil$ and the greatest of them is $\left \lfloor b / n \right \rfloor$, therefore
they form an arithmetic progression of known length and difference:

$$
\sum_{h=a}^{b} \left[ n | h \right] h =
\frac{1}{2} n
\left( \left \lceil a / n \right \rceil + \left \lfloor b / n \right \rfloor \right)
\left( \left \lfloor b / n \right \rfloor - \left \lceil a / n \right \rceil + 1 \right)
$$

Therefore, for a fixed number of digits $\delta_N$ and for whatever interval $\left(a_i, b_i \right)$, if $a_i$ and $b_i$ have the same number of digits, we have an efficient algorithm to find all invalid numbers, as follows:

- Determine the divisors of $\delta_N$, excluding $\delta_N$ itself. Those are our candidate values for $k$.

- For each of them, find the sum of all multiples of $\mu(\delta_N, k)$ ranging from $a_i$ to $b_i$.

It would be tempting to say that we're done, but there's a minor problem. Numbers that are multiples of _both_ $\mu(\delta_N, k_1)$ and $\mu(\delta_N, k_2)$ for
two different values of $k_1, k_2$ have been counted twice. But it's easy enough to solve: we can perform inclusion-exclusion on the set of the divisors of
$\delta_N$.

To conclude, there's a second minor problem: $a_i$ and $b_i$ could have a different number of digits. But this is easily solved: we can "split" our interval in
two subintervals each of which only contains numbers with the same number of digits. For example, we can solve the problem separately for $(80, 99)$ and
$(100, 110)$ and combine the results to obtain the final result for the interval $(80, 110)$.

### Complexity analysis

The algorithm is very efficient and terminates instantly on normal hardware. It's interesting to analyze its theoretical complexity. Because we can cache intermediate results, it becomes difficult to give a straightforward analysis on a complex case, as it would require making assumptions on the distribution of input intervals. We'll simplify the analysis, assuming to run our algorithm on an interval bounded by two integers $(a, b)$ of length $n$ and $m$ respectively.
In other words, $10^{n-1} \le a \lt 10^n$, $10^{m-1} \le b \lt 10^m$.

We'll split the interval in $n-m$ subintervals as per our trick, each representing a different number of digits.
For the subinterval that represents $h$ digits, we'll have to perform a number of operations that's proportional to... $2^{d(h)}$ where $d(h)$ is the number of
divisors of $h$. This is because inclusion-exclusion effectively requires iterating on the powerset:

$$
\sum_{i=1}^{d(h)} \binom{d(h)}{i} = 2^{d(h)}-1
$$

Asymptotic bounds for the number of divisors of $h$ are well-known in literature. In particular, by Dirichlet's divisor problem:

$$
\sum_{i=1}^{x} d(i) = x \log x + x (2 \gamma - 1) + O(\sqrt{x})
$$

The time complexity $T(n)$ of our algorithm is therefore bounded by:

$$
T(n) = \sum_{i=m}^{n} 2^{d(i)} \le \sum_{i=1}^{n} 2^{d(i)}
$$

By Jensen's inequality:

$$
\frac{T(n)}{n} = \frac{1}{n} \sum_{i=1}^{n} 2^{d(i)} \ge 2 ^ {\frac{1}{n} \sum_{i=1}^n d(i)}
$$

Which means:

$$
T(n) \in O(n 2^{\log n}) = O(n^2)
$$

Which doesn't seem great in theory, although the complexity is probably dominated by numbers divisible by a lot of small primes.

In [3]:
@cache
def sum_multiples_of(a, b, n):
    lo = (a - 1) // n + 1
    hi = b // n
    return n * (hi - lo + 1) * (lo + hi) // 2


@cache
def magic_multiple(l, n):
    return (10 ** l - 1) // (10 ** n - 1)


@cache
def invalid_ids(a, b):
    da = len(str(a))
    db = len(str(b))

    low = (
        sum_multiples_of(a, min(10**da-1, b), magic_multiple(da, da//2))
        if da % 2 == 0 else 0
    )

    if da == db:
        return low

    high = (
        sum_multiples_of(max(10**(db-1), a), b, magic_multiple(db, db//2))
        if db % 2 == 0 else 0
    )

    return low + high + sum(
        invalid_ids(max(10**(n-1), a), min(10**n-1, b))
        for n in range(da+1, db)
    )


@cache
def invalid_ids_2(a, b):
    da = len(str(a))
    db = len(str(b))

    da_divisors = {j for j in range(1, da) if da%j == 0}
    low = sum(
        (-1) ** (h + 1) * sum(
            sum_multiples_of(
                a, min(10**da-1, b),
                lcm(*(magic_multiple(da, u) for u in l))
            )
            for l in combinations(da_divisors, h)
        )
        for h in range(1, len(da_divisors)+1)
    )

    if da == db:
        return low

    db_divisors = {j for j in range(1, db) if db%j == 0}
    high = sum(
        (-1) ** (h + 1) * sum(
            sum_multiples_of(
                max(10**(db-1), a), b,
                lcm(*(magic_multiple(db, u) for u in l))
            )
            for l in combinations(db_divisors, h)
        )
        for h in range(1, len(db_divisors)+1)
    )

    return low + high + sum(
        invalid_ids_2(max(10**(n-1), a), min(10**n-1, b))
        for n in range(da+1, db)
    )


ids = [tuple(map(int, x.split('-'))) for x in aocin(2).split(',')]

A = sum(invalid_ids(a, b) for a, b in ids)
assert A == 5398419778

A = sum(invalid_ids_2(a, b) for a, b in ids)
assert A == 15704845910

## [Day 3: Lobby](https://adventofcode.com/2025/day/3)

In [4]:
def max_voltage(battery, banks=2):
    current = []
    for n, u in enumerate(battery):
        while (
            current
            and current[-1] < u
            and len(battery) - n + len(current) > banks
        ):
            current.pop()
        if len(current) < banks:
            current.append(u)
    return int(''.join(current))


batteries = aocin(3).split()

A = sum(max_voltage(b) for b in batteries)
assert A == 17087

A = sum(max_voltage(b, banks=12) for b in batteries)
assert A == 169019504359949